In [ ]:
set.seed(0)

In [ ]:
dataFull = read.csv("heart_failure.csv", header=TRUE)

In [ ]:
head(dataFull)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
1,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
6,90,1,47,0,40,1,204000,2.1,132,1,1,8,1


In [ ]:
# Split data for training and testing
testRows = sample(nrow(dataFull), 0.2*nrow(dataFull))
testRows

[1] 167 129 270 187  85 277 263  79 213  37 105 217 165  89  42 111  20  44  70
[20] 121  40 172  25 248 198  39 160  14 130  45  22 206 230 193 104 255 265 103
[39]  13 288 176 110  84 262  29 141 252 150 221 156 108  48  33 297 149  31 102
[58] 145  73

In [ ]:
dataTest = dataFull[testRows, ]
head(dataTest)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
167,53,0,196,0,60,0,220000,0.7,133,1,1,134,0
129,61,0,248,0,30,1,267000,0.7,136,1,1,104,0
270,40,0,582,1,35,0,222000,1.0,132,1,0,244,0
187,50,0,582,0,50,0,153000,0.6,134,0,0,172,1
85,59,1,280,1,25,1,302000,1.0,141,0,0,78,1
277,70,0,618,0,35,0,327000,1.1,142,0,0,245,0


In [ ]:
dataTrain = dataFull[-testRows, ]
head(dataTrain)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death_event
,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
1,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
6,90,1,47,0,40,1,204000,2.1,132,1,1,8,1


In [ ]:
model2 <- glm(death_event ~ age+ejection_fraction+serum_creatinine+time, family=binomial, data=dataTrain)

In [ ]:
summary(model2)


Call:
glm(formula = death_event ~ age + ejection_fraction + serum_creatinine + 
    time, family = binomial, data = dataTrain)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2150  -0.5301  -0.2213   0.3404   2.3149  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        0.558699   1.238732   0.451   0.6520    
age                0.047097   0.018601   2.532   0.0113 *  
ejection_fraction -0.085596   0.018781  -4.558 5.17e-06 ***
serum_creatinine   0.835901   0.196491   4.254 2.10e-05 ***
time              -0.021201   0.003405  -6.227 4.76e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 293.21  on 239  degrees of freedom
Residual deviance: 164.90  on 235  degrees of freedom
AIC: 174.9

Number of Fisher Scoring iterations: 6


In [ ]:
 # Save model
  cat('Saving model...\n')
  model_path <- file.path("./", 'model.rds')
  saveRDS(model2, file = model_path)

Saving model...


In [ ]:
model = readRDS("model.rds")

In [ ]:
pred.model2 <- predict(model, newdata = dataTest, type = "response")

In [ ]:
predictions <- data.frame(pred.model2, 1-pred.model2)

In [ ]:
colnames(predictions)<-c("postive","negative")

In [ ]:
head(predictions)

,postive,negative
,<dbl>,<dbl>
167,0.012911375,0.9870886
129,0.319520071,0.6804799
270,0.007463053,0.9925369
187,0.010865992,0.9891340
85,0.593822707,0.4061773
277,0.031830085,0.9681699


In [ ]:
return(predictions)

In [ ]:
init <- function(...) {
    require(caret)
    require(glmnet)
}

read_input_data <- function(){
    dataFull = read.csv("heart_failure.csv", header=TRUE)
    return(dataFull)
}

load <- function(){
    model = readRDS("model.rds")
    return(model)
}

transform <- function(data) {
    data[is.na(data)] <- 0
    # Split data for training and testing
    testRows = sample(nrow(data), 0.2*nrow(data))
    dataTest = data[testRows, ]
    return(dataTest)
}

score <- function(data, model){
    pred.model <- predict(model, newdata = dataTest, type = "response")
    predictions <- data.frame(pred.model, 1-pred.model)
    colnames(predictions)<-c("postive","negative")
    return(predictions)
}

In [ ]:
init()

Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Loading required package: glmnet

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘glmnet’”


In [ ]:
data = read_input_data()

In [ ]:
model = load()

In [ ]:
transformed_data = transform(data)

In [ ]:
predictions = score(transformed_data, model)

In [ ]:
predictions

,postive,negative
,<dbl>,<dbl>
167,0.0129113747,0.98708863
129,0.3195200706,0.68047993
270,0.0074630529,0.99253695
187,0.0108659917,0.98913401
85,0.5938227074,0.40617729
277,0.0318300849,0.96816992
263,0.0885276804,0.91147232
79,0.4430084949,0.55699151
213,0.0498424202,0.95015758
